# Long short term memory

We previously explored RNNs, neural networks that are able to propagate some hidden state through a rolled out version of itself. A major problem with RNNs is exploding or vanishing gradients. Gradient clipping solves the exploding gradient problem, but the vanishing gradient problem is harder to solve. LSTMs propose a different architecture which benefits from a hidden state like RNNs, but mitigates the vanishing gradient problem. Another issue RNNs have is that the hidden state often forgets information from a while ago in the sequence and is more biased towards more recent tokens. LSTMs also address this issue with their gated structure. 


In [324]:
from torch import nn
import torch 
import numpy as np
import pandas as pd
from torch.nn.utils.rnn import PackedSequence
import torch.nn.functional
from torch.utils.data import random_split
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import AG_NEWS
from torch import nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

## LSTM model defintion
The goal here is to build a substitute for the torch.nn.rnn.lstm module. This should be able to handle packed sequences and batched data the same way the source code for that module does. For now, does not need to have multiple layers or be bidirectional.

In [325]:
class lstm(nn.Module): 
    def __init__(self, input_size, hidden_dim, output_dim=1) -> None:
        super().__init__()
        self.input_dim = input_size
        self.hidden_dim  = hidden_dim
        
        self.forget_gate = nn.Sequential(
            nn.Linear(input_size+hidden_dim, hidden_dim),
            nn.Sigmoid()
        )
        self.input_gate = nn.Sequential(
            nn.Linear(input_size+hidden_dim, hidden_dim),
            nn.Sigmoid()
        )
        self.input_node = nn.Sequential(
            nn.Linear(input_size+hidden_dim, hidden_dim),
            nn.Tanh()
        )
        self.output_gate = nn.Sequential(
            nn.Linear(input_size+hidden_dim, hidden_dim),
            nn.Sigmoid()
        )
        self.tanh = nn.Tanh()

        # this output layer can be fancier if needed by the use case
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, h_in=None, c_in=None):

        if isinstance(x, PackedSequence):
            input, batch_sizes, sorted_indices, unsorted_indices = x
            max_batch_size = batch_sizes[0]
            if h_in is None: 
                h_in = self.init_h(max_batch_size, x)
                c_in = self.init_h(max_batch_size, x)
        
            data_offset = 0
            outputs = []
            for batch_size in batch_sizes:
                current_input = input[data_offset:data_offset + batch_size]
                data_offset += batch_size
                current_input = current_input.unsqueeze(0)
                if batch_size < max_batch_size: 
                    h_in[:,batch_size:,:] = 0
                    c_in[:,batch_size:,:] = 0
                    pad_size = max_batch_size - batch_size
                    # Create a tensor of zeros with the padding size
                    padding = torch.zeros(1, pad_size, self.hidden_dim, device=current_input.device)
                    # Concatenate the padding to the current input
                    current_input = torch.cat([current_input, padding], dim=1)
                
                combined = torch.cat([current_input, h_in], dim=2)
                
                i_gate_output = self.input_gate(combined)
                i_node_output = self.input_node(combined)
                o_gate_output = self.output_gate(combined)
                f_gate_output = self.forget_gate(combined)

                c_out = (f_gate_output * c_in) + (i_node_output * i_gate_output)

                h_out = self.tanh(c_out) * o_gate_output
                out = self.output_layer(h_out)
                h_in = h_out
                c_in = c_out
                outputs.append(out)

                # Handle decreasing batch siz
            output_packed = PackedSequence(outputs, batch_sizes, sorted_indices, unsorted_indices)
            return output_packed, (h_out, c_out)
        
        else:
            
            for t in range(x.size(1)):
                input = x[:,t,:]
                combined = torch.cat([x, h_in], dim=2)
                        
                i_gate_output = self.input_gate(combined)
                i_node_output = self.input_node(combined)
                o_gate_output = self.output_gate(combined)
                f_gate_output = self.forget_gate(combined)

                c_out = (f_gate_output * c_in) + (i_node_output * i_gate_output)
                h_out = self.tanh(c_out) * o_gate_output
                out = self.output_layer(h_out)
                h_in = h_out 
                c_in = c_out
            
        return out, (h_out, c_out)
    
    
    def init_h(self, batch_size, sequence_length):
        #zero initialization 
        #alternatives include but not limited to Xavier/Kaiminh initialization
        return torch.zeros(batch_size, sequence_length, self.hidden_dim, requires_grad=False)

In [326]:
class newsLSTM(nn.Module): 
    def __init__(self, vocab_size, embed_size, hidden_size) -> None:
        super(newsLSTM, self).__init__()
        
        self.encoder = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.hidden_size = hidden_size 
        self.lstm = nn.LSTM(embed_size, 
                           hidden_size,
                           batch_first=True)
        
        self.hidden2label = nn.Linear(hidden_size, 4)
        self.dropoutLayer = nn.Dropout(p=0.5)


    def forward(self, x, x_len):
        embedded = self.encoder(x)
        x_packed = nn.utils.rnn.pack_padded_sequence(embedded, x_len, batch_first=True, enforce_sorted=False)
        output, (h_t, c_t) = self.lstm(x_packed)  # Pass the initial hidden state 'h' to the RNN   
        # hidden = self.dropout(torch.cat((h_t[-2,:,:], h_t[-1,:,:]), dim=1))     
        hidden = self.dropoutLayer(h_t.squeeze())
        
        # Linear layer and softmax
        label_space = self.hidden2label(hidden)
        
        return label_space
    

We repeat the exact same news classification task performed by in the bidirectionalRNN subdir

In [327]:
train_iter = AG_NEWS(split='train')

# Convert to list to enable random splitting
train_dataset = list(train_iter)

#80-20 train-val split 
train_size = int(len(train_dataset) * 0.8)  
val_size = len(train_dataset) - train_size  
train_data, val_data = random_split(train_dataset, [train_size, val_size])

tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

VOCAB_SIZE = 5000

# Build vocab based on the train_data
train_data_iter = (text for _, text in train_data)
vocab = build_vocab_from_iterator(yield_tokens(train_data_iter), specials=["<unk>"], max_tokens=VOCAB_SIZE)
vocab.set_default_index(vocab["<unk>"])

In [328]:
train_data[0]

(3,
 'Audit of Venezuelan vote backs Chavez An audit of Venezuela #39;s referendum results has confirmed that President Hugo Chavez won the poll fairly and found no evidence to support the opposition #39;s claims of widespread vote-rigging, international observers said ')

In [329]:
vocab(['word', 'probably', 'unknown', 'gibberish'])

[2199, 1699, 4681, 0]

In [330]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [331]:
vocab.lookup_tokens([4999])

['muqtada']

In [332]:
def collate_batch(batch):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    label_list, text_list, lengths = [], [], []
    
    # Sort the batch in the descending order
    batch.sort(key=lambda x: len(x[1]), reverse=True)
    
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        lengths.append(processed_text.size(0))
        
    label_list = torch.tensor(label_list, dtype=torch.int64)
    lengths = torch.tensor(lengths, dtype=torch.int64)
    
    # Pad sequences
    text_list = pad_sequence(text_list, batch_first=True)
    
    return label_list.to(device), text_list.to(device), lengths

In [333]:
train_loader = DataLoader(train_data, batch_size = 8, shuffle = True, collate_fn = collate_batch)
val_loader = DataLoader(val_data, batch_size = 8, shuffle = False, collate_fn = collate_batch)

In [334]:
batch = next(iter(train_loader))

# Inspect the shape of the input data
input_data = batch[1]  # Assuming the input data is the first element of the batch
input_shape = input_data.shape[0]

In [335]:
batch[1].shape

torch.Size([8, 70])

In [336]:
len(batch)

3

In [337]:
a =  torch.ones(5, 50)
a =  torch.ones(5, 50)
a =  torch.ones(5, 50)

In [338]:
m = nn.Softmax(dim=1)
input = torch.randn(2, 3)
output = m(input)

In [339]:
input

tensor([[ 0.3639,  0.2005, -0.1079],
        [ 2.1012, -0.9496, -2.1717]])

In [340]:
output

tensor([[0.4044, 0.3434, 0.2523],
        [0.9423, 0.0446, 0.0131]])

In [341]:
input_shape

8

In [342]:
print(batch[0])
print(batch[1].shape)

tensor([0, 1, 0, 1, 1, 2, 3, 1], device='cuda:0')
torch.Size([8, 70])


In [343]:
LEARNING_RATE = 1e-3
BATCH_SIZE = 128
NUM_EPOCHS = 10
DROPOUT = 0.5
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 128
BIDIRECTIONAL = True
HIDDEN_DIM = 128
NUM_LAYERS = 2
OUTPUT_DIM = 4

In [344]:
model = newsLSTM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [345]:
def train(model, train_loader, val_loader, loss_function, optim, epochs, device):
    
    for epoch in range(epochs):
        losses = [] #group losses for loss visualization 
        running_loss = 0.0
        model.train()
        print("Epoch %d / %d" % (epoch+1, epochs))
        print("-"*10)
    
        for i, batch_data in enumerate(train_loader):
            
            model.train()
            (y, x, x_size) = batch_data
            y_pred = model(x, x_size.cpu())
            #print(f"y_pred: {y_pred}, y_target: {y}")
            loss = loss_function(y_pred, y)
            optim.zero_grad()
            loss.backward()
            optim.step()
            
            running_loss += loss.item()
            losses.append(loss)
            
        model.eval()
        val_loss = 0.0
        print("Epoch: {}, train loss: {:.4f}".format(epoch+1, running_loss/len(losses)))
        with torch.no_grad():
            for i, batch_data in enumerate(val_loader):
                (y, x, x_size) = batch_data
                y, x, x_size = y.to(device), x.to(device), x_size.to(device)
                
                y_pred = model(x, x_size.cpu())
                loss = loss_function(y_pred, y)
                
                val_loss += loss.item()
        
        print("Epoch: {}, validation loss: {:.4f}".format(epoch+1, val_loss/len(val_loader)))

In [346]:
train(model, train_loader, val_loader, torch.nn.functional.cross_entropy, optimizer, NUM_EPOCHS, DEVICE)

Epoch 1 / 10
----------


KeyboardInterrupt: 

# Notes: 

Something about the handling of decreasing batch sizes and packed sequences makes our lstm from scratch inefficient at learning when compared to the native torch version. However, working through the implementation and the bugs that came with it helped me better understand the internals of the lstm. What's especially impressive is that using a single layered unidirectional LSTM is much more effective for learning on this task than a 2 layered bidirectional RNN (see `RecurrentNetworks/bidirectionalRNN`)

Below, we still make use of our hand crafted LSTM for a non batched, non-packed use case where it should not suffer from its inefficiencies.

In [347]:
def train_scratch_model(model, data_loader, loss_function, optim, epochs, device, scheduler, start_decay):
    
    losses = [] #group losses for loss visualization 
    
    for epoch in range(epochs):
        running_loss = 0 
        print("Epoch %d / %d" % (epoch+1, epochs))
        print("-"*10)
        if (epoch > start_decay): 
            scheduler.step()
        

        for i, (x, y) in enumerate(data_loader):
            h_s = model.init_h(x.shape[0], x.shape[1]).to(device) 
            c_s = model.init_h(x.shape[0], x.shape[1]).to(device) 
            
            x = x.to(device)
            y = y.to(device)
            
            y_pred, (h_out, c_out) = model(x, h_s, c_s)
            y_pred = y_pred[:, -1, :]
            
            h_s = h_out
            c_s = c_out
            
            loss = loss_function(y_pred, y) 
            running_loss+=loss.item()
            
            optim.zero_grad()
            loss.backward(retain_graph=True) #backprop 
            optim.step() #update weights
  
        losses.append((running_loss / i))
        print("Step: {}/{}, current Epoch loss: {:.4f}".format(i, len(data_loader), (running_loss / i)))
        
    return losses

In [348]:
def load_data(stock, sequence_length):
    # Convert pandas dataframe to numpy array
    data_raw = stock.to_numpy()

    data = []

    # Loop over the stock data to generate sequences of 'sequence_length' consecutive data points
    # This is done because RNNs learn to predict data in a sequence from past sequence
    for index in range(len(data_raw) - sequence_length):
        data.append(data_raw[index: index + sequence_length])

    data = np.array(data)
    set_size = int(data.shape[0])

    # Generate the train data
    # x_train is all sequences excluding the last data point from each sequence
    x = data[:,:-1,:]
    # y_train is the last data point from each sequence
    y = data[:,-1,:]
    

    # Return the train and test data
    return [x, y]

In [349]:
ibm_df = pd.read_csv('IBM.csv')
ibm_df['Date'] = pd.to_datetime(ibm_df['Date'])
ibm_df.set_index('Date',inplace=True)
ibm_df = ibm_df[['Close']]

# Get the number of rows in the DataFrame
num_rows = ibm_df.shape[0]

# Compute the split point
split_point = int(num_rows*0.8)

# Split the DataFrame
train_df = ibm_df.iloc[:split_point]
test_df = ibm_df.iloc[split_point:]

x_train, y_train = load_data(train_df, 10) 
x_test, y_test = load_data(test_df, 10) 

In [350]:
print(train_df.head()) #to ensure no shuffling accidentally occurred 

            Close
Date             
2006-01-03  82.06
2006-01-04  81.95
2006-01-05  82.50
2006-01-06  84.95
2006-01-09  83.73


In [351]:
print(test_df.tail())

             Close
Date              
2017-12-22  152.50
2017-12-26  152.83
2017-12-27  153.13
2017-12-28  154.04
2017-12-29  153.42


In [352]:
print("Train input data shape: {}, target shape: {}".format(x_train.shape, y_train.shape)) 

Train input data shape: (2406, 9, 1), target shape: (2406, 1)


In [353]:
print("Test input data shape: {}, target shape: {}".format(x_test.shape, y_test.shape)) 

Test input data shape: (594, 9, 1), target shape: (594, 1)


In [354]:
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor)
y_test = torch.from_numpy(y_test).type(torch.Tensor)

In [355]:
train_set = torch.utils.data.TensorDataset(x_train,y_train)
test_set = torch.utils.data.TensorDataset(x_test,y_test)

In [356]:
NUM_EPOCHS = 1000
BATCH_SIZE = 32 
LEARNING_RATE = 3e-4
INPUT_DIM = 1 
OUTPUT_DIM = 1 
HIDDEN_DIM = 32 

In [357]:
train_loader = torch.utils.data.DataLoader(dataset=train_set, 
                                           batch_size=BATCH_SIZE, 
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=test_set, 
                                          batch_size=BATCH_SIZE, 
                                          shuffle=False)

In [358]:
model = lstm(INPUT_DIM, HIDDEN_DIM)

In [359]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
DECAY_FACTOR = 0.1
DECAY_EPOCHS = 20
START_DECAY_EPOCH = 50
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=DECAY_EPOCHS, gamma=DECAY_FACTOR)

In [360]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [361]:
model = model.to(device)

In [362]:
losses = train_scratch_model(model=model, data_loader=train_loader, loss_function=torch.nn.MSELoss(size_average=True), optim=optimizer, epochs=NUM_EPOCHS, device=device, scheduler=scheduler, \
    start_decay=START_DECAY_EPOCH)

Epoch 1 / 1000
----------
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])


/home/aryaman.pandya/ml_accel/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: torch.Size([32, 9, 1])
x size: 

KeyboardInterrupt: 